In [0]:
import tensorflow
from tensorflow.keras.datasets import boston_housing
(train_data, train_targets), (test_data, test_targets) =boston_housing.load_data()

57344/57026 [==============================] - 0s 0us/step


In [0]:
train_data[0]

array([  1.23247,   0.     ,   8.14   ,   0.     ,   0.538  ,   6.142  ,
        91.7    ,   3.9769 ,   4.     , 307.     ,  21.     , 396.9    ,
        18.72   ])

In [0]:
mean = train_data.mean(axis=0)
train_data = train_data - mean
std = train_data.std(axis=0)
train_data /= std
test_data -= mean
test_data /= std

In [0]:
train_data[0]

array([-0.27224633, -0.48361547, -0.43576161, -0.25683275, -0.1652266 ,
       -0.1764426 ,  0.81306188,  0.1166983 , -0.62624905, -0.59517003,
        1.14850044,  0.44807713,  0.8252202 ])

In [0]:
from keras import models
from keras import layers
def build_model():
  model = models.Sequential()
  model.add(layers.Dense(64, activation='relu',
    input_shape=(train_data.shape[1],)))
  model.add(layers.Dense(64, activation='relu'))
  model.add(layers.Dense(1))
  model.compile(optimizer='rmsprop', loss='mse', metrics=['mse'])
  return model

In [0]:
import numpy as np
k=4
num_val_samples = len(train_data) // k
num_epochs = 100
all_scores = []
for i in range(k):
  print('processing fold #', i)
  val_data = train_data[i * num_val_samples: (i + 1) * num_val_samples]
  val_targets = train_targets[i * num_val_samples: (i + 1) * num_val_samples]
  partial_train_data = np.concatenate([train_data[:i * num_val_samples],train_data[(i + 1) * num_val_samples:]],axis=0)
  partial_train_targets = np.concatenate(
        [train_targets[:i * num_val_samples],
        train_targets[(i + 1) * num_val_samples:]],
        axis=0)
  model = build_model()
  model.fit(partial_train_data, partial_train_targets,
  epochs=num_epochs, batch_size=1, verbose=1)
  val_mse, val_mae = model.evaluate(val_data, val_targets, verbose=1)
  all_scores.append(val_mae)

processing fold # 0
Epoch 1/100
303/303 [==============================] - 1s 3ms/step - loss: 177.4031 - mean_squared_error: 177.4031
Epoch 2/100
303/303 [==============================] - 1s 3ms/step - loss: 27.7197 - mean_squared_error: 27.7197
Epoch 3/100
303/303 [==============================] - 1s 2ms/step - loss: 22.0651 - mean_squared_error: 22.0651
Epoch 4/100
303/303 [==============================] - 1s 3ms/step - loss: 18.7842 - mean_squared_error: 18.7842
Epoch 5/100
303/303 [==============================] - 1s 2ms/step - loss: 17.8463 - mean_squared_error: 17.8463
Epoch 6/100
303/303 [==============================] - 1s 2ms/step - loss: 15.9653 - mean_squared_error: 15.9653
Epoch 7/100
303/303 [==============================] - 1s 3ms/step - loss: 15.3339 - mean_squared_error: 15.3339
Epoch 8/100
303/303 [==============================] - 1s 2ms/step - loss: 14.2868 - mean_squared_error: 14.2868
Epoch 9/100
303/303 [==============================] - 1s 2ms/step - loss:

In [0]:
all_scores

In [0]:
np.mean(all_scores)

In [0]:
num_epochs = 500
all_mae_histories = []
for i in range(k):
  print('processing fold #', i)
  val_data = train_data[i * num_val_samples: (i + 1) * num_val_samples]
  val_targets = train_targets[i * num_val_samples: (i + 1) * num_val_samples]
  partial_train_data = np.concatenate(
      [train_data[:i * num_val_samples],
      train_data[(i + 1) * num_val_samples:]],axis=0)
  partial_train_targets = np.concatenate(
      [train_targets[:i * num_val_samples],
      train_targets[(i + 1) * num_val_samples:]],axis=0)
  model = build_model()
  history = model.fit(partial_train_data, partial_train_targets,
  validation_data=(val_data, val_targets),
  epochs=num_epochs, batch_size=1, verbose=1)
  mae_history = history.history['val_mean_absolute_error']
  all_mae_histories.append(mae_history)
